In [1]:
import random
from dataclasses import replace

from replenishment import ArticleSimulationConfig, ReorderPointPolicy, simulate_replenishment_for_articles

{'article-1001': SimulationSummary(total_demand=97, total_fulfilled=87, total_backorders=10, fill_rate=0.8969072164948454, average_on_hand=16.333333333333332, holding_cost=78.4, stockout_cost=35.0, ordering_cost=19.0, total_cost=132.4), 'article-1002': SimulationSummary(total_demand=72, total_fulfilled=72, total_backorders=0, fill_rate=1.0, average_on_hand=28.5, holding_cost=136.8, stockout_cost=0.0, ordering_cost=19.0, total_cost=155.8)}


In [2]:
from replenishment import simulate_replenishment_with_aggregation, optimize_aggregation_windows
)
aggregated_result.summary

candidate_windows = [1, 2, 3]
optimized_windows = optimize_aggregation_windows(articles, candidate_windows)
print({article_id: result.window for article_id, result in optimized_windows.items()})

{'article-1001': 2, 'article-1002': 2}


In [3]:
from replenishment import ArticleSimulationConfig, ForecastBasedPolicy, simulate_replenishment_for_articles

{'article-2001': SimulationSummary(total_demand=124, total_fulfilled=124, total_backorders=0, fill_rate=1.0, average_on_hand=20.5, holding_cost=98.4, stockout_cost=0.0, ordering_cost=57.0, total_cost=155.4), 'article-2002': SimulationSummary(total_demand=93, total_fulfilled=93, total_backorders=0, fill_rate=1.0, average_on_hand=19.0, holding_cost=91.2, stockout_cost=0.0, ordering_cost=57.0, total_cost=148.2)}


In [4]:
from replenishment import optimize_service_level_factors, optimize_aggregation_and_service_level_factors
    forecast_articles,
)
print({article_id: (result.window, result.service_level_factor) for article_id, result in optimized_joint.items()})

{'article-2001': 0.5, 'article-2002': 0.5}
{'article-2001': (2, 1.5), 'article-2002': (2, 0.5)}


In [5]:
from replenishment import ArticleSimulationConfig, PointForecastOptimizationPolicy, PercentileForecastOptimizationPolicy, simulate_replenishment_for_articles

{'article-2101': SimulationSummary(total_demand=111, total_fulfilled=111, total_backorders=0, fill_rate=1.0, average_on_hand=18.833333333333332, holding_cost=79.1, stockout_cost=0.0, ordering_cost=57.0, total_cost=136.1)}
{'article-2201': SimulationSummary(total_demand=78, total_fulfilled=78, total_backorders=0, fill_rate=1.0, average_on_hand=3.8333333333333335, holding_cost=13.799999999999999, stockout_cost=0.0, ordering_cost=57.0, total_cost=70.8)}


In [6]:
from replenishment import ForecastCandidatesConfig, optimize_forecast_targets, optimize_aggregation_and_forecast_targets, simulate_replenishment
    )
    for article_id, config in candidate_configs.items()
}
print({article_id: result.summary for article_id, result in final_results.items()})

# Joint optimization: aggregation window + forecast target.
candidate_windows = [1, 2, 3]
optimized_joint_targets = optimize_aggregation_and_forecast_targets(
    candidate_configs,
    candidate_windows=candidate_windows,
)
print({article_id: (result.window, result.target) for article_id, result in optimized_joint_targets.items()})

{'article-3001': '45-low', 'article-3002': '45-high'}
{'article-3001': SimulationSummary(total_demand=120, total_fulfilled=120, total_backorders=0, fill_rate=1.0, average_on_hand=8.833333333333334, holding_cost=42.400000000000006, stockout_cost=0.0, ordering_cost=57.0, total_cost=99.4), 'article-3002': SimulationSummary(total_demand=84, total_fulfilled=74, total_backorders=10, fill_rate=0.8809523809523809, average_on_hand=2.3333333333333335, holding_cost=8.4, stockout_cost=40.0, ordering_cost=57.0, total_cost=105.4)}
{'article-3001': (2, '45-high'), 'article-3002': (2, '45-high')}


In [7]:
final_results

{'article-3001': SimulationResult(snapshots=[InventorySnapshot(period=0, starting_on_hand=30, demand=18, received=0, ending_on_hand=12, backorders=0, order_placed=19, on_order=19), InventorySnapshot(period=1, starting_on_hand=31, demand=20, received=19, ending_on_hand=11, backorders=0, order_placed=18, on_order=18), InventorySnapshot(period=2, starting_on_hand=29, demand=19, received=18, ending_on_hand=10, backorders=0, order_placed=19, on_order=19), InventorySnapshot(period=3, starting_on_hand=29, demand=21, received=19, ending_on_hand=8, backorders=0, order_placed=20, on_order=20), InventorySnapshot(period=4, starting_on_hand=28, demand=22, received=20, ending_on_hand=6, backorders=0, order_placed=20, on_order=20), InventorySnapshot(period=5, starting_on_hand=26, demand=20, received=20, ending_on_hand=6, backorders=0, order_placed=20, on_order=20)], summary=SimulationSummary(total_demand=120, total_fulfilled=120, total_backorders=0, fill_rate=1.0, average_on_hand=8.833333333333334, h

In [8]:
optimized_targets

{'article-3001': PercentileForecastOptimizationResult(target='45-low', policy=PercentileForecastOptimizationPolicy(forecast=[18, 19, 18, 19, 20, 20], lead_time=1), simulation=SimulationResult(snapshots=[InventorySnapshot(period=0, starting_on_hand=30, demand=18, received=0, ending_on_hand=12, backorders=0, order_placed=19, on_order=19), InventorySnapshot(period=1, starting_on_hand=31, demand=20, received=19, ending_on_hand=11, backorders=0, order_placed=18, on_order=18), InventorySnapshot(period=2, starting_on_hand=29, demand=19, received=18, ending_on_hand=10, backorders=0, order_placed=19, on_order=19), InventorySnapshot(period=3, starting_on_hand=29, demand=21, received=19, ending_on_hand=8, backorders=0, order_placed=20, on_order=20), InventorySnapshot(period=4, starting_on_hand=28, demand=22, received=20, ending_on_hand=6, backorders=0, order_placed=20, on_order=20), InventorySnapshot(period=5, starting_on_hand=26, demand=20, received=20, ending_on_hand=6, backorders=0, order_plac

In [9]:
results["article-1001"].snapshots[:3]


[InventorySnapshot(period=0, starting_on_hand=40, demand=15, received=0, ending_on_hand=25, backorders=0, order_placed=0, on_order=0),
 InventorySnapshot(period=1, starting_on_hand=25, demand=10, received=0, ending_on_hand=15, backorders=0, order_placed=50, on_order=50),
 InventorySnapshot(period=2, starting_on_hand=15, demand=25, received=0, ending_on_hand=0, backorders=10, order_placed=0, on_order=50)]